In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Pandas for working with data in DataFrame structures
import pandas as pd

# Planetary Computer Tools for accessing and working with Planetary Computer data
import pystac
import pystac_client
import odc
import planetary_computer as pc
from odc.stac import stac_load
pc.settings.set_subscription_key('e9c294b01b2f483082124b8329af819f') # INSERT YOUR KEY HERE

# Requests for making HTTP requests
import requests

# Rich for creating rich-text tables
from rich.table import Table

# Itertools for creating an iterator cycling through a sequence indefinitely
from itertools import cycle

# TQDM for creating progress bars
from tqdm import tqdm

# Data Science
import pandas as pd
import numpy as np

In [2]:
def get_sentinel_data(latlong, time_slice):
    '''
    Returns VV and VH values for a given latitude and longitude 
    Attributes:
    latlong - A tuple with 2 elements - latitude and longitude
    time_slice - Timeframe for which the VV and VH values have to be extracted
    '''

    latlong = latlong.replace('(', '').replace(')', '').replace(' ', '').split(',')
    lat, lon = map(float, latlong)
    
    #box_size_deg = 0.0004  # Surrounding box in degrees, yields approximately 5x5 pixel region
    box_size_deg = 0.0125
    
    min_lon = lon - box_size_deg/2
    min_lat = lat - box_size_deg/2
    max_lon = lon + box_size_deg/2
    max_lat = lat + box_size_deg/2
    
    bbox_of_interest = (min_lon, min_lat, max_lon, max_lat)
    time_of_interest = time_slice

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
    )
    search = catalog.search(
        collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_interest
    )
    items = list(search.get_all_items())
    
    # Define the pixel resolution for the final product
    # Define the scale according to our selected crs, so we will use degrees
    resolution = 10  # meters per pixel 
    scale = resolution / 111320.0  # degrees per pixel for crs=4326 
    
    # Load the data using Open Data Cube
    data = stac_load(items, bands=["vv", "vh"], patch_url=pc.sign, bbox=bbox_of_interest, crs="EPSG:4326", resolution=scale)
    
    # Calculate the statistics of VH and VV
    vh_min = data.vh.min().values.item()
    vh_max = data.vh.max().values.item()
    vh_median = np.median(data.vh).item()

    vv_min = data.vv.min().values.item()
    vv_max = data.vv.max().values.item()
    vv_median = np.median(data.vv).item()

    # Calculate the mean of the data across the sample region
    mean = data.mean(dim=['latitude', 'longitude']).compute()
    
    # Calculate RVI
    dop = (mean.vv / (mean.vv + mean.vh))
    m = 1 - dop
    rvi = (np.sqrt(dop)) * ((4 * mean.vh) / (mean.vv + mean.vh))

    # Calculate additional RVI statistics
    rvi_min = rvi.min().values.item()
    rvi_max = rvi.max().values.item()
    rvi_median = np.median(rvi).item()
    
    # Calculate the averaged values
    vh_avg = mean.vh.mean().values.item()
    vv_avg = mean.vv.mean().values.item()
    rvi_avg = rvi.mean().values.item()
    
    return vh_avg, vv_avg, rvi_avg, rvi_min, rvi_max, rvi_median, vh_min, vh_max, vh_median, vv_min, vv_max, vv_median


In [3]:
#Reading the coordinates for the submission
test_file = pd.read_csv('challenge_1_submission_template.csv')
test_file.head()

,Latitude and Longitude,Class of Land
0,"(10.18019073690894, 105.32022315786804)",NaN
1,"(10.561107033461816, 105.12772097986661)",NaN
2,"(10.623790611954897, 105.13771401411867)",NaN
3,"(10.583364246115156, 105.23946127195805)",NaN
4,"(10.20744446668854, 105.26844107128906)",NaN


In [4]:
## Get Sentinel-1-RTC Data
time_slice = "2020-03-20/2020-03-21"
assests = ['vh','vv','rvi']
vh_vv_rvi = []
for coordinates in tqdm(test_file['Latitude and Longitude']):
    vh_vv_rvi.append(get_sentinel_data(coordinates,time_slice))
submission_vh_vv_rvi_data = pd.DataFrame(np.vstack(vh_vv_rvi),columns =['vh', 'vv', 'RVI', 'RVI(min)', 'RVI(max)', 'RVI(median)', 'vh(min)', 'vh(max)', 'vh(median)', 'vv(min)', 'vv(max)', 'vv(median)'])

100%|██████████| 250/250 [33:26<00:00,  8.03s/it]


In [5]:
submission_vh_vv_rvi_data.head()

,vh,vv,RVI,RVI(min),RVI(max),RVI(median),vh(min),vh(max),vh(median),vv(min),vv(max),vv(median)
0,0.022501,0.149149,0.488522,0.459407,0.517636,0.488522,0.001122,0.536283,0.017607,0.003845,1.086427,0.132135
1,0.031486,0.151358,0.628084,0.614971,0.641197,0.628084,0.000900,0.401623,0.024936,0.003047,1.123161,0.134585
2,0.039603,0.136223,0.791227,0.770222,0.812231,0.791227,0.003468,0.313000,0.035717,0.006694,3.830926,0.117729
3,0.042935,0.147232,0.794516,0.789525,0.799506,0.794516,0.000641,4.416507,0.014040,0.001529,9.210921,0.053367
4,0.027665,0.156761,0.554118,0.526656,0.581579,0.554118,0.001192,3.002685,0.023604,0.004182,7.247988,0.137934


In [6]:
def combine_two_datasets(dataset1,dataset2):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    '''
    data = pd.concat([dataset1,dataset2], axis=1)
    return data

In [7]:
export_data = combine_two_datasets(test_file,submission_vh_vv_rvi_data)
export_data.head()

,Latitude and Longitude,Class of Land,vh,vv,RVI,RVI(min),RVI(max),RVI(median),vh(min),vh(max),vh(median),vv(min),vv(max),vv(median)
0,"(10.18019073690894, 105.32022315786804)",NaN,0.022501,0.149149,0.488522,0.459407,0.517636,0.488522,0.001122,0.536283,0.017607,0.003845,1.086427,0.132135
1,"(10.561107033461816, 105.12772097986661)",NaN,0.031486,0.151358,0.628084,0.614971,0.641197,0.628084,0.000900,0.401623,0.024936,0.003047,1.123161,0.134585
2,"(10.623790611954897, 105.13771401411867)",NaN,0.039603,0.136223,0.791227,0.770222,0.812231,0.791227,0.003468,0.313000,0.035717,0.006694,3.830926,0.117729
3,"(10.583364246115156, 105.23946127195805)",NaN,0.042935,0.147232,0.794516,0.789525,0.799506,0.794516,0.000641,4.416507,0.014040,0.001529,9.210921,0.053367
4,"(10.20744446668854, 105.26844107128906)",NaN,0.027665,0.156761,0.554118,0.526656,0.581579,0.554118,0.001192,3.002685,0.023604,0.004182,7.247988,0.137934


In [8]:
#Dumping the predictions into a csv file.
export_data.to_csv("export_data.csv",index = False)